In [1]:
import pandas as pd
import re
import io
import os
import glob
import time
from unicodedata import normalize
from sqlalchemy import create_engine
import psycopg2 as ps
from sqlalchemy import text

#database information to connect
host = 'ds4ateam77.cxdzzcrtbiby.us-east-2.rds.amazonaws.com'
port = 5432
user = 'postgres'
password = '12345678'
database = 'teate'
def runQuery(sql):
    engine=create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}',max_overflow=30)
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())


In [2]:
#Formulas to use further
def formatcolumn(text,flag = True):
    text = text.lower()
    text = re.sub('<.*?>', '', text)
    text = re.sub(':.*?:', '', text)
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    text = normalize( 'NFC', text)
    if flag:
        text = re.sub('[^a-z0-9 ]', '', text)
    else:
        text = re.sub('[^a-z0-9]', '', text)
    return text

In [3]:
data = pd.read_csv('upload/Teate_clean_w_sep.csv',encoding = 'unicode_escape',dtype='str')  

In [4]:
data.columns = [formatcolumn(val,False) for val in data.columns]

In [5]:
_N='\n'
_R='\r'
_E=''
data= data.replace(_N,_E, regex=True)
data= data.replace(_R,_E, regex=True)


In [6]:
data

,pedido,fechapedido,tienda,nombretienda,direccintienda,fabricante,nombrefabricante,posped,material,nombrematerial,...,fechacreacintienda,zona,nombrezonacomercial,ruta,nombreruta,comuna,barrio,poblacin,descuentoteat,valorcupnteat
0,1000067562,10/1/19,20000207,Tienda NN4,CL 83 B 2 B 73,5000069,COMERCIALIZADORA PROCON S.A,100,705,Caf Aguila Roja 50gr,...,5/3/17,NaN,NaN,L,Lunes,Comuna 6,Floralia,Santiago de cali,NaN,NaN
1,1000067562,10/1/19,20000207,Tienda NN4,CL 83 B 2 B 73,5000069,COMERCIALIZADORA PROCON S.A,201,706,Caf Aguila Roja 125gr,...,5/3/17,NaN,NaN,L,Lunes,Comuna 6,Floralia,Santiago de cali,NaN,NaN
2,1000067563,10/1/19,20003910,Tienda Mixta Reina,CL 20 B # 40 B SUR Ð 06 ESQ,5000020,ORGANIZACION ROA S.A,100,57,Roa Arroba 25 unid. x 500 g,...,1/31/19,NaN,NaN,W,Mircoles,COMUNA 99,Terranova VI,Jamund,NaN,NaN
3,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000081,KELLOGG DE COLOMBIA S.A,1200,884,Choco Krispis 35gr,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
4,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000081,KELLOGG DE COLOMBIA S.A,1300,885,Zucaritas 35gr,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
5,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000056,COLOMBINA S.A,300,1048,Chocobreak & Cookies,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
6,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000056,COLOMBINA S.A,400,1049,Chocobreak Tradicional,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
7,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000057,MONDELEZ COLOMBIA S.A.S,201,766,Club Social x 234 Gr Original,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
8,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000026,COMESTIBLES ALDOR S.A.S,100,100,Mangoos,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
9,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000081,KELLOGG DE COLOMBIA S.A,1501,892,Zucaritas 130gr,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN


In [33]:
connDB = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

data =data.reset_index()
data

,index,pedido,fechapedido,tienda,nombretienda,direccintienda,fabricante,nombrefabricante,posped,material,...,fechacreacintienda,zona,nombrezonacomercial,ruta,nombreruta,comuna,barrio,poblacin,descuentoteat,valorcupnteat
0,0,1000067562,10/1/19,20000207,Tienda NN4,CL 83 B 2 B 73,5000069,COMERCIALIZADORA PROCON S.A,100,705,...,5/3/17,NaN,NaN,L,Lunes,Comuna 6,Floralia,Santiago de cali,NaN,NaN
1,1,1000067562,10/1/19,20000207,Tienda NN4,CL 83 B 2 B 73,5000069,COMERCIALIZADORA PROCON S.A,201,706,...,5/3/17,NaN,NaN,L,Lunes,Comuna 6,Floralia,Santiago de cali,NaN,NaN
2,2,1000067563,10/1/19,20003910,Tienda Mixta Reina,CL 20 B # 40 B SUR Ð 06 ESQ,5000020,ORGANIZACION ROA S.A,100,57,...,1/31/19,NaN,NaN,W,Mircoles,COMUNA 99,Terranova VI,Jamund,NaN,NaN
3,3,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000081,KELLOGG DE COLOMBIA S.A,1200,884,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
4,4,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000081,KELLOGG DE COLOMBIA S.A,1300,885,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
5,5,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000056,COLOMBINA S.A,300,1048,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
6,6,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000056,COLOMBINA S.A,400,1049,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
7,7,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000057,MONDELEZ COLOMBIA S.A.S,201,766,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
8,8,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000026,COMESTIBLES ALDOR S.A.S,100,100,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN
9,9,1000067564,10/1/19,20000024,Tienda La 73,Calle 73 # 3 A N 95,5000081,KELLOGG DE COLOMBIA S.A,1501,892,...,12/2/16,NaN,NaN,L,Lunes,COMUNA 6,Floralia,Santiago de cali,NaN,NaN


In [51]:
conn = connDB.raw_connection()
cursor = conn.cursor()
zconn.commit()
conn.close()

In [52]:
len(data)
data.columns

Index(['index', 'pedido', 'fechapedido', 'tienda', 'nombretienda',
       'direccintienda', 'fabricante', 'nombrefabricante', 'posped',
       'material', 'nombrematerial', 'cantidaddepedido', 'um',
       'valorunitariopedido', 'mon', 'valortotalpedido', 'mon1',
       'valortotalpeddctoiva', 'mon2', 'entrega', 'factura', 'fechafactura',
       'posfac', 'ctdfacturada', 'um3', 'um4', 'valorunitariofactura', 'mon5',
       'valortotalfactura', 'mon6', 'valortotcondscto', 'mon7', 'mr',
       'denominacinmotivorechazo', 'fechamovil', 'horamovil',
       'valoringresopedido', 'mon8', 'valoringresofactur', 'mon9', 'ce',
       'ntatmvil', 'cupndscto', 'valorcupndescuento', 'mon10',
       'fechacreacintienda', 'zona', 'nombrezonacomercial', 'ruta',
       'nombreruta', 'comuna', 'barrio', 'poblacin', 'descuentoteat',
       'valorcupnteat'],
      dtype='object')

In [54]:
S_=0
range(S_,len(data),D_)

range(0, 662877, 10000)

In [58]:
S_=0
D_=10000

start_time = time.time()

print("Start at "+str(start_time))
for n in range(S_,len(data),D_):
    S_=n
    E_=n+D_
    df2=data[(data.index>=S_) & (data.index<=E_)]

    engine=connDB
    conn = engine.raw_connection()
    cur = conn.cursor()
    
    output = io.StringIO()
    df2.to_csv(output, sep='|', header=False, index=False)
    output.seek(0)
    cur.copy_from(output, 'fact_orders_S', null="" , sep='|') # null values become ''
    conn.commit()
    
    Comp = str(round((n+D_)/len(data)*100,1))
    Records = str(S_) +" to "+ str(E_) +" "+str(len(df2))
    print(Records+" "+Comp+"pct--- %s seconds ---" % round((time.time() - start_time),4) + " "+str(time.time()))
print('Done 100%')
print("--- %s seconds ---" % (time.time() - start_time))
conn.close()


Start at 1601691860.450523
0 to 10000 10001 1.5pct--- 26.0227 seconds --- 1601691886.4732175
10000 to 20000 10001 3.0pct--- 46.2347 seconds --- 1601691906.6852458
20000 to 30000 10001 4.5pct--- 62.3266 seconds --- 1601691922.7771554
30000 to 40000 10001 6.0pct--- 91.5946 seconds --- 1601691952.045143
40000 to 50000 10001 7.5pct--- 105.3158 seconds --- 1601691965.766386
50000 to 60000 10001 9.1pct--- 118.0617 seconds --- 1601691978.5122151
60000 to 70000 10001 10.6pct--- 137.9835 seconds --- 1601691998.4340105
70000 to 80000 10001 12.1pct--- 151.857 seconds --- 1601692012.307537
80000 to 90000 10001 13.6pct--- 169.5876 seconds --- 1601692030.0381086
90000 to 100000 10001 15.1pct--- 186.68 seconds --- 1601692047.1304965
100000 to 110000 10001 16.6pct--- 197.7884 seconds --- 1601692058.2389662
110000 to 120000 10001 18.1pct--- 219.2254 seconds --- 1601692079.6759377
120000 to 130000 10001 19.6pct--- 241.2811 seconds --- 1601692101.7316265
130000 to 140000 10001 21.1pct--- 255.1668 seconds